In [370]:
import kagglehub
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np

Access the appendix of this paper (https://www.irjet.net/archives/V3/i4/IRJET-V3I4213.pdf) to read more about the meaning of each columns.

In [371]:
# Download latest version
path = kagglehub.dataset_download("jpacse/datasets-for-churn-telecom")

# List files in the directory to find the CSV
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {path}")

# Construct path to the CSV file
csv_path = os.path.join(path, csv_files[0])

# Load the CSV into a DataFrame
df = pd.read_csv(csv_path)

In [372]:
df.head().style.set_properties()

In [373]:
# look through the datatype of each column

df.dtypes

CustomerID                     int64
Churn                         object
MonthlyRevenue               float64
MonthlyMinutes               float64
TotalRecurringCharge         float64
DirectorAssistedCalls        float64
OverageMinutes               float64
RoamingCalls                 float64
PercChangeMinutes            float64
PercChangeRevenues           float64
DroppedCalls                 float64
BlockedCalls                 float64
UnansweredCalls              float64
CustomerCareCalls            float64
ThreewayCalls                float64
ReceivedCalls                float64
OutboundCalls                float64
InboundCalls                 float64
PeakCallsInOut               float64
OffPeakCallsInOut            float64
DroppedBlockedCalls          float64
CallForwardingCalls          float64
CallWaitingCalls             float64
MonthsInService                int64
UniqueSubs                     int64
ActiveSubs                     int64
ServiceArea                   object
H

# Data Quality Assessment and Preprocessing

In [392]:
# Check for duplicated CustomerID 

print(df['CustomerID'].duplicated().any())

False


In [374]:
# IncomeGroup's data type is int64 when it should be categorical

df['IncomeGroup'] = df['IncomeGroup'].astype('category')

In [376]:
unique_vals = {
    col: df[col].unique()
    for col in df.select_dtypes(include=['object', 'category']).columns
}

# print(unique_vals)

# MaritalStatus and HandsetPrice contains 'Unknown' => convert them to nan
df.replace('Unknown', np.nan, inplace=True)

In [377]:
print(f"Number of unique values for ServiceArea: {df['ServiceArea'].nunique()}")

# There are too many ServiceArea so we will convert them to just include cities
df['ServiceCity'] = df['ServiceArea'].str[:3]

print(f"Number of unique values for ServiceCity: {df['ServiceCity'].nunique()}")


Number of unique values for ServiceArea: 747
Number of unique values for ServiceCity: 57


In [378]:
print(f"Column(s) where there are only 1 unique values: {[col for col in df if df[col].nunique() == 1]}")

# recheck
print(df['Homeownership'].value_counts())

# The value for Homeownership is always 0 so they are not meaningful 
# Drop the column 'Homeownership' from df

df = df.drop('Homeownership', axis=1)

['Homeownership']
Homeownership
Known    33987
Name: count, dtype: int64


In [379]:
# Count total rows in the dataframe
print(f"Total rows in dataframe: {len(df)}") 

# Count the percentage of rows where AgeHH1 == 0
print(f"% of rows in dataframe where AgeHH1 is 0: {round((df['AgeHH1'] == 0).sum() / len(df) * 100, 2)}") 

# Count the percentage of rows where AgeHH1 == 0 AND ChildrenInHH == "No"
print(f"% of rows in dataframe where AgeHH1 is 0 and ChildrenInHH is 'No': {round(((df['AgeHH1'] == 0) & (df['ChildrenInHH'] == 'No')).sum() / len(df) * 100, 2)}") 

# Note: There are households where age of head household is 0 but there are children.
# Secondary sources did not explain why AgeHH1 is 0 so we will assume that these are meant to be null. 
df.loc[df['AgeHH1'] == 0, 'AgeHH1'] = np.nan
print("AgeHH1 = 0 finished converting to null")

Total rows in dataframe: 51047
% of rows in dataframe where AgeHH1 is 0: 27.26
% of rows in dataframe where AgeHH1 is 0 and ChildrenInHH is 'No': 27.03
AgeHH1 = 0 finished converting to null


In [380]:
# (Umayaparvathi and Iyakutti 1070) mentioned that IncomeGroup = 0 indicates missing data
# So, convert IncomeGroup = 0 to nan

df['IncomeGroup'] = df['IncomeGroup'].copy()
df.loc[df['IncomeGroup'] == 0, 'IncomeGroup'] = np.nan

In [381]:
# Create a DataFrame with columns having missing values
null_df = df.isnull().sum()[df.isnull().sum() > 0].to_frame(name='null_count').reset_index()

# Total rows count
total_len = len(df)

# Add missing value proportion (%)
null_df['null_proportion(%)'] = null_df['null_count'] / total_len * 100
print(f"{null_df} \n")

# Rename columns for clarity
null_df.columns = ['column', 'null_count', 'null_proportion(%)']

# List of columns with missing values
null_cols = null_df['column'].tolist()

non_numerical = []
skewness_list = []

# Calculate skewness for numeric columns; track non-numeric columns
for col_name in null_cols:
    if pd.api.types.is_numeric_dtype(df[col_name]):
        skewness_list.append(df[col_name].skew())
    else:
        non_numerical.append(col_name)

# Create a copy of null_df for numeric columns only (drop non-numerical)
numerical_null_df = null_df[~null_df['column'].isin(non_numerical)].copy()

# Add skewness values to the filtered DataFrame
numerical_null_df['Skewness'] = skewness_list

print(numerical_null_df)

                    index  null_count  null_proportion(%)
0          MonthlyRevenue         156            0.305601
1          MonthlyMinutes         156            0.305601
2    TotalRecurringCharge         156            0.305601
3   DirectorAssistedCalls         156            0.305601
4          OverageMinutes         156            0.305601
5            RoamingCalls         156            0.305601
6       PercChangeMinutes         367            0.718945
7      PercChangeRevenues         367            0.718945
8             ServiceArea          24            0.047015
9                Handsets           1            0.001959
10          HandsetModels           1            0.001959
11   CurrentEquipmentDays           1            0.001959
12                 AgeHH1       14826           29.043822
13                 AgeHH2         909            1.780712
14            IncomeGroup       12835           25.143495
15           HandsetPrice       28982           56.775129
16          Ma

In [382]:
# null_df shows that there are data with a high skewness value, so median imputations for NaN values would be prefered. 
# avoid imputing for columns where null proportion is high to reduce risk of distorting data

to_be_imputed = null_df.loc[null_df['null_proportion(%)'] < 30, 'column'].tolist()
print(to_be_imputed)

for col_name in to_be_imputed:
    # If numeric, impute with the median. Else (if categorical), use mode.
    if pd.api.types.is_numeric_dtype(df[col_name]):
        median_val = df[col_name].median(skipna=True)
        df[col_name] = df[col_name].fillna(median_val)
    else:
        mode_val = df[col_name].mode()
        df[col_name] = df[col_name].fillna(mode_val[0])

# Check  
null_df_2 = df.isnull().sum()[df.isnull().sum() > 0].to_frame(name='null_count').reset_index()
print(f"\n Null values were NOT imputed: {null_df_2.equals(null_df)}")

['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge', 'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls', 'PercChangeMinutes', 'PercChangeRevenues', 'ServiceArea', 'Handsets', 'HandsetModels', 'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2', 'IncomeGroup', 'ServiceCity']

 Null values were NOT imputed: False


In [383]:
# Impute missing values in columns where the null proportion is higher than 30%.
# Method to deal with these columns come from (Fujo et al. 188)

to_be_imputed = null_df.loc[null_df['null_proportion(%)'] > 30, 'column'].tolist()

for col_name in to_be_imputed:
    if pd.api.types.is_numeric_dtype(df[col_name]):
        # Impute numerical columns with mean (Fujo et al. 188)
        df[col_name] = df[col_name].fillna(df[col_name].mean())
        print(f"This numerical column was imputed with mean: {col_name}\n")
    else:
        # Impute categorical columns with most frequent (mode) (Fujo et al. 188)
        mode_val = df[col_name].mode()
        df[col_name] = df[col_name].fillna('Unknown')
        print(f"This categorical column was imputed with mode: {col_name}\n")

This categorical column was imputed with mode: HandsetPrice

This categorical column was imputed with mode: MaritalStatus



In [385]:
def columns_with_mixed_types(df):
    mixed_cols = []
    for col in df.columns:
        # Get unique types in the column (excluding NaNs)
        types = df[col].dropna().map(type).unique()
        if len(types) > 1:
            mixed_cols.append((col, types))
    return mixed_cols

mixed_columns = columns_with_mixed_types(df)
print("Columns with mixed types and their types:")
for col, types in mixed_columns:
    print(f"{col}: {types}")

Columns with mixed types and their types:


In [384]:
# Log-transform columns to address skewness > 1

column_list = df.columns.drop('CustomerID').tolist()

for col_name in column_list:
    if pd.api.types.is_numeric_dtype(df[col_name]):  
        # filter for columns where skewness > 1 and there is no negative values
        if df[col_name].skew() > 1 and df[col_name].min() >= 0: 
            df[col_name] = np.log1p(df[col_name])  # handles 0 safely: log(1 + x)
            print(f"Log-transformed: {col_name}")

Log-transformed: MonthlyMinutes
Log-transformed: DirectorAssistedCalls
Log-transformed: OverageMinutes
Log-transformed: RoamingCalls
Log-transformed: DroppedCalls
Log-transformed: BlockedCalls
Log-transformed: UnansweredCalls
Log-transformed: CustomerCareCalls
Log-transformed: ThreewayCalls
Log-transformed: ReceivedCalls
Log-transformed: OutboundCalls
Log-transformed: InboundCalls
Log-transformed: PeakCallsInOut
Log-transformed: OffPeakCallsInOut
Log-transformed: DroppedBlockedCalls
Log-transformed: CallForwardingCalls
Log-transformed: CallWaitingCalls
Log-transformed: MonthsInService
Log-transformed: UniqueSubs
Log-transformed: ActiveSubs
Log-transformed: Handsets
Log-transformed: HandsetModels
Log-transformed: RetentionCalls
Log-transformed: RetentionOffersAccepted
Log-transformed: ReferralsMadeBySubscriber
Log-transformed: AdjustmentsToCreditRating


In [390]:
df.to_csv('cleaned_data.csv', index=False)